# C/C++ DLL using MinGW-w64 with Python and Excel VBA


## 1 Using C/C++ DLL with Python

The advantage of Python is that it is **flexible and easy** to program. The time it takes to setup a new calulation is therefore short. But for certain types of calculations Python (and any other interpreted language) can be **very slow**. It is particularly iterations over large arrays that is difficult to do efficiently.

Such calculations may be implemented in a compiled language such as C or Fortran. In Python it is relatively easy to call out to libraries with compiled C or Fortran code. 

But before we go ahead and work on optimizing anything, it is always worthwhile to ask.... 

In [ ]:
import seuif97
%timeit seuif97.pt2h(15,535)

In [ ]:
from iapws.iapws97 import IAPWS97
%timeit IAPWS97(P=16.10,T=535.10).h

## ctypes

ctypes is a Python library for calling out to C code.

We manually need to load the library and set properties such as the functions return and argument types.

On the otherhand we do not need to touch the C code at all. 

In [11]:
%%file funs.c

#include <stdio.h>

void hello();
double dprod(double *x, int n);

void hello()
{
   printf("C says Hello world!\n");
}

double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

Writing funs.c


Compile the C file into a shared library:

In [12]:
!gcc -c -O3 -Wall -fPIC -o funs.o funs.c
!gcc -o libfuns.dll -shared funs.o

In [13]:
%%file makefile

CC=gcc
CFLAGS=-O3 -Wall -fPIC -o 

all: libfuns.dll

libfuns.dll: funsobj
	 $(CC) -o libfuns.dll -shared funs.o
	 del funs.o
    
funsobj: ./code/gcc/funs.c
	 $(CC) -c $(CFLAGS) funs.o ./code/gcc/funs.c
     
clean:
	 del libfuns.dll

Overwriting makefile


In [14]:
!make

gcc -c -O3 -Wall -fPIC -o  funs.o ./code/gcc/funs.c
gcc -o libfuns.dll -shared funs.o
del funs.o


The result is a compiled shared library `libfuns.dll`

Now we need to write wrapper functions to access the C library: 

* To load the library we use the **ctypes** package, which included in the Python standard library (with extensions from numpy for passing arrays to C). 

* Then we manually set the types of the argument and return values. 

   * cdll.LoadLibrary

   * .argtypes

   * .restype

In [15]:
%%file funs.py

from ctypes import c_int, c_double,c_void_p,cdll
import numpy as np

_lib = cdll.LoadLibrary('libfuns.dll')
#_lib = np.ctypeslib.load_library('libfuns', '.')

_lib.hello.argtypes = []
_lib.hello.restype  =  c_void_p

_lib.dprod.argtypes = [np.ctypeslib.ndpointer(dtype=np.float), c_int]
_lib.dprod.restype  = c_double

def hello():
    return _lib.hello()

def dprod(x):
    n = len(x)
    x = np.asarray(x, dtype=np.float)
    return _lib.dprod(x, int(n))

Writing funs.py


In [19]:
%%file run_hello_c.py

import funs
funs.hello()

Overwriting run_hello_c.py


In [20]:
!python run_hello_c.py

C says Hello world!


### Product function:

In [24]:
import funs
funs.dprod([2,3,4,5,6]) 

720.0

## 2 C/C++ DLL using MinGW-w64 for Visual Basic Application with Excel

  For Visual Basic applications (or applications in other languages such as Pascal or Fortran) to call functions in a C/C++ DLL, the functions must be exported using the correct calling convention without any name decoration done by the compiler.
  
*  `__stdcall` creates the correct calling convention for the function (the called function cleans up the stack and parameters are passed from right to left)

* ` __declspec(dllexport)` is used on an exported function in a DLL

Below is the example of techniques which facilitate the use of use of MinGW to create DLLs, exporting functions which may be called from Visual Basic Application with Excel. 

#### Step 1: Create your DLL.

VBA can only call `__stdcall` functions, not `__cdecl` functions. So we must export all our functions as `__stdcall`. 

Create a DLL with the following code:

* mathfuns.h

* mathfuns.c

In [21]:
%%file ./code/gcc/mathfuns.h

#ifdef BUILD_DLL
#define DLLPORT __declspec(dllexport)
#else
#define DLLPORT __declspec(dllimport)
#endif

DLLPORT int __stdcall iadd(int a,int b);
DLLPORT float __stdcall fmult(float a, float b);

Overwriting ./code/gcc/mathfuns.h


When you create header files for your DLLs, use

* ` __declspec(dllexport) ` adds the export directive to the object fileworks

* ` __declspec(dllimport)`  on the declarations of the public symbols

In [22]:
%%file ./code/gcc/mathfuns.c

#include "mathfuns.h"

int iadd(int a,int b){
   return a+b;
}

float fmult(float a, float b){
   float product;
   product = a *b;
   return product;
}

Overwriting ./code/gcc/mathfuns.c


In [23]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o libmathfuns.dll mathfuns.o -Wl,--add-stdcall-alias

* -DBUILD_DLL:
   
  * -Dname: Predefine name as a macro, with definition 
  

*  -Wl,option 

   Pass **option** as an option to the **linker**. If option contains commas, it is split into multiple options at the commas.


* --add-stdcall-alias:
   
   the option to the linker: Add aliases without @<n>.

In [33]:
!gcc -c -DBUILD_DLL ./code/gcc/mathfuns.c
!gcc -shared -o libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def

* -static-libgcc

   This option links the GNU libgcc library statically. 
   

* -output-def=libmathfuns.def

    Name of .def file to be created.
    
    **def:** A module-definition  file is a text file containing one or more module statements that describe various attributes of a DLL

In [21]:
%%file makefile

CC=gcc
CFLAGS=-O3 -DBUILD_DLL -Wall -fPIC -o 

all: libmathfuns.dll

libmathfuns.dll: mathfunsobj
	 $(CC) -shared -o libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
	 del mathfuns.o
    
mathfunsobj: ./code/gcc/mathfuns.c
	 $(CC) -c $(CFLAGS) mathfuns.o ./code/gcc/mathfuns.c
     
clean:
	 del libmathfuns.dll

Overwriting makefile


In [22]:
!make

gcc -c -O3 -DBUILD_DLL -Wall -fPIC -o  mathfuns.o ./code/gcc/mathfuns.c
gcc -shared -o libmathfuns.dll -static-libgcc mathfuns.o -Wl,--add-stdcall-alias,-output-def=libmathfuns.def
del mathfuns.o


### Step 2: Call DLL from Excel VBA

* 1) libmathfuns.dll in the default dll path of windows c:\windows\system

* 2) demo-vba.xlsm in ./code/gcc/

* 3)  press "ALT+F11" into VBA

create the module *mathfuns* to library call such as:
```vba
Public Declare PtrSafe Function iadd Lib "libmathfuns" (ByVal a As Integer, ByVal a As Integer) As Integer
Public Declare PtrSafe Function fmult Lib "libmathfuns" (ByVal a As Single, ByVal b As Single) As Single
```
create the module *vbaapi* to call such as:
```
Public Function intadd(ByVal a As Integer, ByVal b As Integer) As Integer
   intadd = iadd(a, b)
End Function
 
Public Function smult(ByVal a As Single, ByVal b As Single) As Single
   smult = fmult(a, b)
End Function
```

* 4)  call in cells
![demo-vba](./code/gcc/demo-vba.jpg)

### 3 call_stdcall DLL from Python

```python
 windll.LoadLibrary
```

In [4]:
%%file mathfuns.py

from ctypes import windll,c_int,c_float

flib = windll.LoadLibrary('libmathfuns.dll')


def iadd(a,b):
    flib.iadd.argtypes = [c_int,c_int]
    flib.iadd.restype  = c_int
    return flib.iadd(a,b)

def fmult(a,b):
    flib.fmult.argtypes = [c_float,c_float]
    flib.fmult.restype  = c_float
    return flib.fmult(a,b)


Writing mathfuns.py


In [5]:
from mathfuns import *
print(iadd(4,6))
print(fmult(5,5))

10
25.0


### 4 Call __stdcall DLL from C 

In [6]:
%%file ./code/gcc/mainmathfuns.c

#include <stdio.h>
#include "mathfuns.h"
 
int main() {
    int ir=iadd(5,6);
    float fr=fmult(2.1,4.1);
    printf("ir: %d \t fr: %.3f",ir,fr);
    return 0;
}

Writing ./code/gcc/mainmathfuns.c


In [8]:
!gcc -c -o mainmathfuns.o ./code/gcc/mainmathfuns.c 
!gcc -o  mainmathfuns.exe mainmathfuns.o -I./code/gcc -L./ -lmathfuns

In [9]:
!mainmathfuns

ir: 11 	 fr: 8.610


In [16]:
%%file makefile

all: mainmathfuns.exe

clean:
	del mainmathfuns.exe

mainmathfuns.exe: mainmathfuns.o ./code/gcc/mathfuns.h 
	gcc -o mainmathfuns mainmathfuns.o -I./code/gcc -L./ -lmathfuns
	del *.o

mainmathfuns.o: ./code/gcc/mainmathfuns.c 
	gcc -c ./code/gcc/mainmathfuns.c 


Overwriting makefile


In [17]:
!make

gcc -o mainmathfuns mainmathfuns.o -I./code/gcc -L./ -lmathfuns
del *.o


In [14]:
!make clean

del mainmathfuns.exe


In [18]:
!mainmathfuns

ir: 11 	 fr: 8.610


### Further reading

* http://docs.python.org/3/library/ctypes.html
* http://scipy-cookbook.readthedocs.io/items/Ctypes.html
* https://docs.scipy.org/doc/numpy/reference/routines.ctypeslib.html


* DLLs in Visual C++ https://msdn.microsoft.com/en-us/library/1ez7dh12.aspx

* Calling DLL Functions from Visual Basic Applications https://msdn.microsoft.com/en-us/library/dt232c9t.aspx


* Getting Started with VBA in Office 2010: https://msdn.microsoft.com/library/office/ee814735(v=office.14)
   
* Excel VBA Programming:  http://www.homeandlearn.org/the_excel_vba_editor.html


